---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [4]:
## mm/dd/yy format
# Ref: https://bolinwu.blog/post/text-mining-in-python-p1-basics-of-regex/
# * Assume all dates in xx/xx/xx format are mm/dd/yy

df1 = df.str.extract(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})').dropna(how='any')
df1.head(10)




/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


,month,day,year
0,03,25,93
1,6,18,85
2,7,8,71
3,9,27,75
4,2,6,96
5,7,06,79
6,5,18,78
7,10,24,89
8,3,7,86
9,4,10,71


In [10]:
# * Assume all dates where year is encoded in only two digits 
# are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)

df2 = df.str.extract(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ,.-/]+(?P<day>\d{2,4})[ ,.-/]+(?P<year>\d{2,4})').dropna(how='any')

df2.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


,month,day,year
194,April,11,1990
195,May,30,2001
196,Feb,18,1994
197,February,18,1981
198,October,11,2013
199,Jan,24,1986
200,July,26,1978
201,December,23,1999
202,May,15,1989
203,September,06,1995


* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

In [15]:
def month_to_num(string):
    m = {
        'jan':1,'feb':2,'mar':3,'apr':4,'may':5,
        'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,
        'nov':11,'dec':12
    }
    s = string.strip()[:3].lower()
    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')

month_to_num('October')

for index,row in df2.iterrows():
    row['month'] = month_to_num(row['month'])

df2.head(10)

,month,day,year
194,4,11,1990
195,5,30,2001
196,2,18,1994
197,2,18,1981
198,10,11,2013
199,1,24,1986
200,7,26,1978
201,12,23,1999
202,5,15,1989
203,9,06,1995


In [17]:
# Change date format of df to dd/mm/yy -> 20 Mar 2009 to 
# 20 March 2009; 20 March. 2009 to 20 March 2009 Format

df3 = df.str.extract(r'(?P<day>\d{1,2})[ ,.-](?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ,.-](?P<year>\d{2,4})').dropna(how='any')


# Change month name to number

for index, row in df3.iterrows():
    row['month'] = month_to_num(row['month'])
df3.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


,day,month,year
125,24,1,2001
126,10,9,2004
127,26,5,1982
128,28,6,2002
129,06,5,1972
130,25,10,1987
131,14,10,1996
132,30,11,2007
133,28,6,1994
134,14,1,1981


In [18]:
# Check for records in below format in df
# Mar 20th, 2009; Mar 21st 2009; Mar 22nd,2009 Format

df.str.extract(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[ ,.-](\d{2,4})(?:th|st|rd), (\d{2,4})').dropna(how='any')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  after removing the cwd from sys.path.


,0,1,2


In [22]:
# No records were found in that format
# Check for records where day is missing
# Feb 2009; Sep 2004; Oct 2010 format

df4 = df.str.extract(r'\D(?P<day> )(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*),?[ ](?P<year>\d{4})').dropna(how='any')

#Convert month alphabetic row to num
for index,row in df4.iterrows():
    row['month'] = month_to_num(row['month'])
print(df4.head(5))

# Find indices where day is missing
mi = [index for index in range(df4.index[0], df4.index[len(df4.index)-1])
     if index not in df4.index]
print('List of indices where day is missing in df \n',mi)

    day month  year
228         9  1985
230         5  1986
231         5  2016
232         7  1977
233         7  1990
List of indices where day is missing in df 
 [229, 235, 245, 247, 250, 251, 256, 266, 270, 276, 281, 283, 285, 294, 299, 306, 308, 318, 321, 324, 330, 333, 340, 341]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """


In [23]:
## Let's checkout some indices where day is missing

df.iloc[229]

't Allergies Sulfa (Sulfonamide Antibiotics) - Renal Toxicity : pt developed acute interstitial nephritis on Bactrim (June 2011)\n'

In [26]:
df.iloc[270]

'May, 2006 Primary Care Doctor:\n'

In [29]:
# Fix characters before month ex: ttMay 2004, eeJune 2019;

df42 = df.str.extract(r'^[\D\. ]?(?P<day>)(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,. ]*(?P<year>\d{4})\D').dropna(how = 'any')

#convert month to no
for index, row in df42.iterrows():
    row['month'] = month_to_num(row['month'])
    
# Find indices where day is missing
mi = [index for index in range(df4.index[0], df4.index[len(df4.index)-1])
     if index not in (df4.index | df42.index)]
print('List of indices where day is missing in df \n',mi)
# df42.head(5)

List of indices where day is missing in df 
 [229, 250, 308, 318, 321, 324, 330, 340]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
# Checkout missing indexes after first search
df.iloc[229]

't Allergies Sulfa (Sulfonamide Antibiotics) - Renal Toxicity : pt developed acute interstitial nephritis on Bactrim (June 2011)\n'

In [31]:
df.iloc[250]
#Date is enclosed in ()

') He endorsed the following hyperarousal symptoms: disturbed sleep (e.g., months with great sleep and then has nights he barely sleeps- mind racing with worries related to job, expecting a child (May 2005)); reckless or self-destructive behavior (e.g., see above); difficulty with concentration; some hypervigilance; and exaggerated startle response triggered by loud noises.\n'

In [32]:
df.iloc[318]
#Date is enclosed in ()

') Paxil (Jan 1978) : sedation\n'

In [34]:
# Fix characters before month ex: (Jan 1978),(May 2005),(June 2011);

df43 = df.str.extract(r'[^]*[\D\. ]?(?P<day>)(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,. ]*(?P<year>\d{4})\D').dropna(how = 'any')


#convert month to no
for index, row in df43.iterrows():
    row['month'] = month_to_num(row['month'])
print(df43.head(5))   
# Find indices where day is missing
mi = [index for index in range(df4.index[0], df4.index[len(df4.index)-1])
     if index not in (df4.index | df42.index | df43.index)]
print('List of indices where day is missing in df \n',mi)


    day month  year
125         1  2001
126         9  2004
127         5  1982
128         6  2002
129         5  1972
List of indices where day is missing in df 
 []


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
# Check dates where day is missing in format 6/2008; 12/2009 Format

df5 = df.str.extract(r'(?:^| |~|[a-z])+-?(?P<day>)(?P<month>\d{1,2})[/-](?P<year>\d{4})').dropna(how = 'any')
df5.head(5)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


,day,month,year
343,,6,1998
344,,6,2005
345,,10,1973
346,,9,2005
347,,03,1980


In [37]:
# Concatenate the extracted dataframes

import numpy as np
import pandas as pd

frames = [df1, df2, df3, df4, df42, df43, df5]
result = pd.concat(frames)
print(result.head(10))

#replace missing values with 1
result = result.replace(r'^\s*$',1,regex=True)
print(result.head(10))

  day month year
0  25    03   93
1  18     6   85
2   8     7   71
3  27     9   75
4   6     2   96
5  06     7   79
6  18     5   78
7  24    10   89
8   7     3   86
9  10     4   71
  day month year
0  25    03   93
1  18     6   85
2   8     7   71
3  27     9   75
4   6     2   96
5  06     7   79
6  18     5   78
7  24    10   89
8   7     3   86
9  10     4   71


In [38]:
df.shape

(500,)

In [40]:
# Final text cleaning of concatenated df

# add leading 0 and add 19 for 2 digit year

for index , row in result.iterrows():
    if len(row['year'])<3:
        row['year'] = '19' + row['year']
    if len(str(row['day'])) == 1:
        row['day'] = '0' + str(row['day'])
    if len(str(row['month'])) == 1:
        row['month'] = '0' + str(row['month'])

print(result.head(10))

  day month  year
0  25    03  1993
1  18    06  1985
2  08    07  1971
3  27    09  1975
4  06    02  1996
5  06    07  1979
6  18    05  1978
7  24    10  1989
8  07    03  1986
9  10    04  1971


In [41]:
# Convert the col to str

result = result.applymap(str)

# Add date column for date as yy/mm/dd

result['combine'] = result['year'] + result['month'] + result['day']
result['combine'] = result['combine'].astype(int)

print(result.dtypes)

day        object
month      object
year       object
combine     int64
dtype: object


In [42]:
print(result.head(10))

  day month  year   combine
0  25    03  1993  19930325
1  18    06  1985  19850618
2  08    07  1971  19710708
3  27    09  1975  19750927
4  06    02  1996  19960206
5  06    07  1979  19790706
6  18    05  1978  19780518
7  24    10  1989  19891024
8  07    03  1986  19860307
9  10    04  1971  19710410


In [43]:
## Sort the combined date

result['combine'] = pd.to_datetime(result['combine'], format = '%Y%m%d')
datesort = pd.DataFrame(pd.to_datetime(result['combine'], format = '%Y%m%d'))

datesort.head(10)


,combine
0,1993-03-25
1,1985-06-18
2,1971-07-08
3,1975-09-27
4,1996-02-06
5,1979-07-06
6,1978-05-18
7,1989-10-24
8,1986-03-07
9,1971-04-10


In [44]:
datesort = datesort.sort_values(ascending = True, by = 'combine')
datesort.reset_index(inplace = True)
datesort.head(5)

,index,combine
0,9,1971-04-10
1,84,1971-05-18
2,2,1971-07-08
3,53,1971-07-11
4,28,1971-09-12


In [13]:
def date_sorter():
    
    regex1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    regex2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
    regex3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    regex4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    regex5 = '(\d{1,2}[/-][1|2]\d{3})'
    regex6 = '([1|2]\d{3})'
    full_regex = '(%s|%s|%s|%s|%s|%s)' %(regex1, regex2, regex3, regex4, regex5, regex6)
    parsed_date = df.str.extract(full_regex)
    parsed_date = parsed_date.iloc[:,0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')
    parsed_date = pd.Series(pd.to_datetime(parsed_date))
    parsed_date = parsed_date.sort_values(ascending=True).index
    return pd.Series(parsed_date.values)

len(date_sorter())

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.


500

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here